In [16]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from prophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
from datetime import datetime

In [26]:
!pip install prophet
     

In [27]:
!pip install fbprophet

^C


In [17]:
#read in csv for
all_bdrm = pd.read_csv("housing_prices_data.csv")
all_bdrm.dropna(inplace=True)
all_bdrm.head()


,RegionID,Neighborhood,State,City,Metro,County,2020-01-31,2020-02-29,2020-03-31,2020-04-30,...,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31,2023-01-31,2023-02-28,BedroomCount
0,118208,South Los Angeles,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,350403.0,354217.0,357681.0,360467.0,...,485481.0,486580.0,486336.0,486796.0,486585.0,487578.0,486809.0,489862.0,489152.0,1
1,268496,Southeast Los Angeles,CA,Los Angeles,"Los Angeles-Long Beach-Anaheim, CA",Los Angeles County,312659.0,314476.0,315822.0,317131.0,...,410623.0,413743.0,416449.0,419431.0,420848.0,422551.0,422515.0,427628.0,430679.0,1
2,273565,East San Jose,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,394852.0,397467.0,399616.0,400423.0,...,465598.0,465629.0,463605.0,460273.0,456309.0,452782.0,449427.0,451437.0,454010.0,1
3,273088,Bullard,CA,Fresno,"Fresno, CA",Fresno County,99098.0,103441.0,107280.0,111859.0,...,154223.0,156623.0,158448.0,159079.0,159166.0,158239.0,157523.0,162523.0,167592.0,1
4,276652,West San Jose,CA,San Jose,"San Jose-Sunnyvale-Santa Clara, CA",Santa Clara County,488197.0,492374.0,495291.0,494556.0,...,552316.0,550898.0,546854.0,542083.0,538433.0,536506.0,535151.0,551396.0,573673.0,1


In [66]:
#chose the first row in as a test 
sla_df = all_bdrm.loc[all_bdrm.index==0]

In [79]:
#transpose data to
transposed_sla = sla_df.transpose()
transposed_sla

,0
RegionID,118208
Neighborhood,South Los Angeles
State,CA
City,Los Angeles
Metro,"Los Angeles-Long Beach-Anaheim, CA"
County,Los Angeles County
2020-01-31,350403.0
2020-02-29,354217.0
2020-03-31,357681.0
2020-04-30,360467.0


In [80]:
# drop unneeded columns
# drop all columns that are not dates
transposed_sla.drop(['RegionID','Neighborhood','State','City','Metro','County','BedroomCount'],axis=0, inplace=True)

In [81]:
#create a column from the index since the index are the dates we are looking at.
transposed_sla['ds'] = transposed_sla.index
transposed_sla

,0,ds
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31
2020-06-30,361653.0,2020-06-30
2020-07-31,362266.0,2020-07-31
2020-08-31,364425.0,2020-08-31
2020-09-30,369158.0,2020-09-30
2020-10-31,374385.0,2020-10-31


In [83]:
#changing the month_date column to datetime data type
transposed_sla['ds'] = pd.to_datetime(transposed_sla['ds'], format="%Y-%m-%d")

,0,ds
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31


In [92]:
# change the price column to y to fit in prophet model
transposed_sla.rename(columns={0 : 'y'}, inplace=True)
transposed_sla.head()

,y,ds
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31


In [93]:
#split data into train and test,
#test data is chosen to hold the last six months since that will be used as the data to check the accuracy
train = transposed_sla.iloc[:len(transposed_sla)-6]
test = transposed_sla.iloc[len(transposed_sla)-6:]

In [94]:
type(train)

pandas.core.frame.DataFrame

In [95]:
train.head()

,y,ds
2020-01-31,350403.0,2020-01-31
2020-02-29,354217.0,2020-02-29
2020-03-31,357681.0,2020-03-31
2020-04-30,360467.0,2020-04-30
2020-05-31,361981.0,2020-05-31


In [ ]:
#train model
# no need to scale data since we are looking at one feeature which is price

In [98]:
model = Prophet()
model

In [99]:
model.fit(train)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

INFO:prophet:n_changepoints greater than number of observations. Using 24.


In [100]:
future = model.make_future_dataframe(periods=6, freq='MS') #MS for monthly, H for hourly
sla_forecast = model.predict(future)

/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [101]:
#view of the forcasted data at the end of the dataframe
sla_forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
33,2022-10-01,496383.430774,492420.906380,494491.940600,495386.493757,497457.527976,-2965.587377,-2965.587377,-2965.587377,-2965.587377,-2965.587377,-2965.587377,0.0,0.0,0.0,493417.843397
34,2022-11-01,497447.221692,492183.646612,499838.870303,493614.178704,501269.402395,-1430.532092,-1430.532092,-1430.532092,-1430.532092,-1430.532092,-1430.532092,0.0,0.0,0.0,496016.689599
35,2022-12-01,498476.696773,498206.456990,513521.790457,490760.143949,506075.477416,7446.313040,7446.313040,7446.313040,7446.313040,7446.313040,7446.313040,0.0,0.0,0.0,505923.009813
36,2023-01-01,499540.487691,496390.573698,521343.538774,486634.816775,511587.781851,9755.756923,9755.756923,9755.756923,9755.756923,9755.756923,9755.756923,0.0,0.0,0.0,509296.244615
37,2023-02-01,500604.278609,496908.336648,533354.012909,481520.797008,517966.473270,15387.539639,15387.539639,15387.539639,15387.539639,15387.539639,15387.539639,0.0,0.0,0.0,515991.818248


In [102]:
sla_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
33,2022-10-01,493417.843397,492420.906380,494491.940600
34,2022-11-01,496016.689599,492183.646612,499838.870303
35,2022-12-01,505923.009813,498206.456990,513521.790457
36,2023-01-01,509296.244615,496390.573698,521343.538774
37,2023-02-01,515991.818248,496908.336648,533354.012909


In [103]:
plot_plotly(model, sla_forecast)


In [104]:
plot_components_plotly(model, sla_forecast)

/Users/rayzapien/opt/anaconda3/lib/python3.9/site-packages/prophet/forecaster.py:896: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [33]:
from statsmodels.tools.eval_measures import rmse

In [107]:
predictions = sla_forecast.iloc[-6:]['yhat']

In [108]:
print("Root Mean Squared Error between actual and  predicted values: ",rmse(predictions,test['y']))
print("Mean Value of Test Dataset:", test['y'].mean())

Root Mean Squared Error between actual and  predicted values:  16234.301990458318
Mean Value of Test Dataset: 487797.0
